## **ETL: Initial Data Transform**

_PRIMERA FASE_

En esta fase, se realizará las primeras y elementales transformaciones que se realizan a los datos inmediatamente después de haber sido leídos desde la fuente principal. También se realizan los desempaquetados de las columnas anidadas.
Finalmente, al final se generará un dataset que será guardado en archivos CSV que serán utilizados por el siguiente notebook: EDA.


In [1]:
## Se importan las librerías necesarias para esta primera fase de transformación de datos
import pandas as pd
import ast
import html
from datetime import datetime

In [2]:
## Se crea una función para leer archivos JSON anidados y con codificación de página UTF-8
def df_read_nestedjson(file_withpath):
    data = []
    with open(file_withpath, "r", encoding="UTF-8") as file:
        for line in file:
            data.append(ast.literal_eval(line))
    return pd.DataFrame(data)

Antes de proceder a cargar los archivos JSON que se encontraban comprimidos en formato GZ
se procedió a descomprimirlos con el utilitario Winrar y también se cambiaron sus nombres para que sean
prácticamente identificables:

- GAMES:      output_steam_games.json       ->    steam_games.json
- REVIEWS:    australian_user_reviews.json  ->    user_reviews.json
- ITEMS:      australian_user_items.json    ->    user_items.json

y se colocaron en la carpeta: Data1_SteamServer

In [4]:

## Se cargan los datos ORIGINALES en formato JSON a sus respectivos DataFrames. Los 2 últimos son anidados.
df_games  = pd.read_json('Data1_SteamServer\steam_games.json', lines=True)
df_reviews = df_read_nestedjson('Data1_SteamServer\\user_reviews.json')
df_items   = df_read_nestedjson('Data1_SteamServer\\users_items.json')

In [5]:
## Se limpian los Dataframes eliminando los registros completamente vacíos
df_games.dropna(how="all", inplace=True)
df_reviews.dropna(how="all", inplace=True)
df_items.dropna(how="all", inplace=True)

In [6]:
## Se limpian los Dataframes eliminando los registros duplicados
df_games.drop_duplicates(subset=['title'], inplace=True)
df_reviews.drop_duplicates(subset=['user_id'], inplace=True)
df_items.drop_duplicates(subset=['user_id'], inplace=True)

In [7]:
## Se generan las respectivas copias de seguridad de los dataframes creados
df_games_bkup   = df_games.copy()
df_reviews_bkup = df_reviews.copy()
df_items_bkup   = df_items.copy()

_SEGUNDA FASE_


In [8]:
## Recuperación de los datos guardados en la copia de seguridad
df_games   = df_games_bkup.copy()
df_reviews = df_reviews_bkup.copy()
df_items   = df_items_bkup.copy()

In [9]:
print("df_games = [" + ", ".join(["'" + col + "'" for col in df_games.columns]) + "]")
print("df_reviews = [" + ", ".join(["'" + col + "'" for col in df_reviews.columns]) + "]")
print("df_items = [" + ", ".join(["'" + col + "'" for col in df_items.columns]) + "]")

df_games = ['publisher', 'genres', 'app_name', 'title', 'url', 'release_date', 'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id', 'developer']
df_reviews = ['user_id', 'user_url', 'reviews']
df_items = ['user_id', 'items_count', 'steam_id', 'user_url', 'items']


In [10]:
## ELIMINACIÓN DE COLUMNAS NIVEL 1

## Después de revisar el diccionario de datos y tras la evaluación de los campos necesarios para los modelos
## decidí eliminar columnas PRIMARIAS innecesarias por estar duplicadas o contener datos irrevelantes

df_games   = df_games.drop(columns= ['publisher','title','url','reviews_url','specs','price','early_access','developer'])
df_reviews = df_reviews.drop(columns=['user_url'])
df_items   = df_items.drop(columns=['steam_id','user_url'])

In [11]:
## DESEMPAQUETADO DE CAMPOS ANIDADOS SEGUNDO NIVEL

'''df_reviews'''

# Desempaquetado de columna anidada REVIEWS, y desglose por USER_ID
reviews_nested = []
for index, row in df_reviews.iterrows():
  user_id = row['user_id']
  for reviews_dict in row['reviews']:
    review_data = {'user_id': user_id, **reviews_dict}
    reviews_nested.append(review_data)
df_reviews = pd.DataFrame(reviews_nested)

'''df_items'''

# Desempaquetado de columna anidada ITEMS, y desglose por USER_ID
items_nested = []
for index, row in df_items.iterrows():
  user_id   = row['user_id']
  items_count = row['items_count']
  for items_dict in row['items']:
    item_data = {'user_id': user_id, 'items_count': items_count, **items_dict}
    items_nested.append(item_data)
df_items = pd.DataFrame(items_nested)

In [12]:
# GUARDAR LOS DATOS EN ARCHIVOS CSV
df_games.to_csv('Data2_ETL_out\\ETL_games.csv', index=False)
df_reviews.to_csv('Data2_ETL_out\\ETL_reviews.csv', index=False)
df_items.to_csv('Data2_ETL_out\\ETL_items.csv', index=False)